## librairies

In [50]:
import pandas as pd
import re
from tqdm import tqdm
import requests
from datetime import datetime
import sqlite3
import hashlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# exploration

## let's go

In [3]:
url = 'https://www.hellowork.com/searchoffers/getsearchfacets?k=Data+analyst&k_autocomplete=Data+analyst&l=france&ray=all&cod=all&d=all&p=1&mode=scroll&alert=%20&timestamp=1701361414591'
print(url)

https://www.hellowork.com/searchoffers/getsearchfacets?k=Data+analyst&k_autocomplete=Data+analyst&l=france&ray=all&cod=all&d=all&p=1&mode=scroll&alert=%20&timestamp=1701361414591


In [4]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    "Referer" : "www.hellowork.com"
}

response = requests.get(url, headers=headers)
print(response)

<Response [200]>


In [ ]:
page = 1
periode = "all"  #pr 24 h c'est "h"
local = "france"
keyword = "Data analyst"
params = {
    "k": keyword,

    "l": local,
    "ray": "all",
    "cod": "all",
    "d": "all",
    "p": page,
    "mode": "scroll",
    "alert": " ",
    "timestamp": 1701361414591
}

response = requests.get("https://www.hellowork.com/searchoffers/getsearchfacets", params=params)
print(response.status_code)
print(response.url)

403
https://www.hellowork.com/searchoffers/getsearchfacets?k=Data+analyst&l=france&ray=all&cod=all&d=all&p=1&mode=scroll&alert=+&timestamp=1701361414591


In [ ]:
def api_helloworld(periode = "all", #pr periode  24 h c'est "h"
                   local = "france",
                   keyword = "Data Analyst"):
  page = 1
  params = {
      "k": keyword,

      "l": local,
      "ray": "all",
      "cod": "all",
      "d": "all",
      "p": page,
      "mode": "scroll",
      "alert": " ",
      "timestamp": 1701361414591
  }
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    "Referer" : "www.hellowork.com"
  }

  response = requests.get("https://www.hellowork.com/searchoffers/getsearchfacets",
                          params=params, headers = headers)
  while page <= 30:

    return response

In [ ]:
api_helloworld()

<Response [200]>

In [ ]:
api_helloworld(keyword = "data engineer")

<Response [200]>

# code clean

In [35]:
def api_helloworld(periode="h", local="france", keyword="Data Analyst"):
    results = []
    page = 1
    while True:
        params = {
            "k": keyword,
            "l": local,
            "ray": "all",
            "cod": "all",
            "d": periode,
            "p": page,
            "mode": "scroll",
            "alert": " ",
            "timestamp": 1701361414591
        }
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            "Referer": "www.hellowork.com"
        }

        response = requests.get("https://www.hellowork.com/searchoffers/getsearchfacets",
                               params=params, headers=headers)
        response_json = response.json()
        if "Results" not in response_json or not response_json["Results"]:
            break
        for result in tqdm(response_json["Results"]):
            publication_date = result["PublishDate"]
            match = re.search(r'(\d{4}-\d{2}-\d{2})', publication_date)
            formatted_date = match.group(0) if match else publication_date
            lien = result["UrlOffre"]
            lien_origine = (f'https://www.hellowork.com{lien}')
            results.append({
                "entreprise": result["CompanyName"],
                "publication": formatted_date,
                "poste": result["OfferTitle"],
                "contrat": result["ContractType"],
                "contrat_type": result["Criterions"][3]["Label"],
                "teletravail": result["Telework"],
                "salaire": result["DisplayedSalary"],
                "profil": result["Profile"],
                "description": result["Description"],
                "ville": result["Localisation"],
                "lien": lien_origine,
                "source": result["ResponseUrl"]
        })

        page += 1

    df = pd.DataFrame(results)
    df.reset_index(inplace=True)
    return df

In [36]:
df = api_helloworld()

100%|██████████| 21/21 [00:00<00:00, 4501.71it/s]


# nettoyage

In [37]:
df.to_csv('/content/drive/MyDrive/wild code school/Projets/Projet_3/scrapping/cadre_emploi/df_hellowork_not_clean.csv', index= False)

In [29]:
df = pd.read_csv('/content/drive/MyDrive/wild code school/Projets/Projet_3/scrapping/cadre_emploi/df_hellowork_not_clean.csv')

In [38]:
df.columns

Index(['index', 'entreprise', 'publication', 'poste', 'contrat',
       'contrat_type', 'teletravail', 'salaire', 'profil', 'description',
       'ville', 'lien', 'source'],
      dtype='object')

In [39]:
def clean(df):
    df['site_annonce'] = 'hellowork'
    df['publication'] = pd.to_datetime(df['publication'],dayfirst=True, errors='coerce')
    df['publication'] = df['publication'].dt.strftime('%d-%m-%Y')
    df = df.apply(lambda x: x.str.lower() if x.name != 'publication' else x, axis=1)
    return df

In [40]:
df = clean(df)

<ipython-input-39-63e5fa784ca1>:3: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['publication'] = pd.to_datetime(df['publication'],dayfirst=True, errors='coerce')


In [41]:
def generate_id(row, columns):
    # Concaténer les valeurs des colonnes sélectionnées en une seule chaîne
    value = ''.join(str(row[col]) for col in columns)
    # Générer un hachage MD5 de cette chaîne
    return hashlib.md5(value.encode()).hexdigest()

In [42]:
df['id'] = df.apply(lambda row: generate_id(row, ["entreprise", "poste" , "contrat" , "ville"]), axis=1)

In [43]:
colonnes_finales = [
        'id', 'site_annonce', 'entreprise', 'publication', 'poste',
        'contrat','teletravail',
        'profil','description', 'ville','lien'
    ]
df = df.reindex(columns=colonnes_finales)

In [44]:
print(df.head().to_markdown())

|    | id                               | site_annonce   | entreprise                                                     | publication   | poste            | contrat    | teletravail         | profil                                                                                                                                                                                                                                   | description                                                                                                                                                                                                                                                                                                                                                                           | ville                | lien                                                  |
|---:|:---------------------------------|:---------------|:---------------------------------------------------------

In [46]:
df['publication'] = pd.to_datetime(df['publication'],dayfirst=True)

In [49]:
df.to_csv("/content/drive/MyDrive/wild code school/Projets/Projet_3/hellowork_clean.csv",index=False)

In [51]:
def db_file_storage(db_name, table_name, df):

    # Connexion ou création du fichier .db
    conn = sqlite3.connect(f"{db_name}.db")

    # Enregistrer le DataFrame dans le fichier
    df.to_sql(name=table_name, con=conn, if_exists='append', index=False)

    # Fermer la connexion
    conn.close()

In [52]:
db_file_storage("hellowork","df_clean",df)